# Get weight values from .tflite file

- [https://www.tensorflow.org/api_docs/python/tf/lite/Interpreter](https://www.tensorflow.org/api_docs/python/tf/lite/Interpreter)

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
import datetime

In [3]:
print('TensorFlow version: ', tf.__version__)

TensorFlow version:  1.15.3


## Load .tflite file

In [4]:
tfl_path = 'mnist_float.tflite'

In [5]:
interp = tf.lite.Interpreter(model_path=tfl_path)
interp.allocate_tensors()

In [6]:
interp.get_input_details()

[{'name': 'batch_normalization_v1_input',
  'index': 14,
  'shape': array([ 1, 28, 28,  1]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0)}]

## Show tensor info

- __t__ is list of tensor & type of dict ('name', 'index', 'shape', 'dtype', 'quantization')

In [7]:
for t in interp.get_tensor_details():
    print('name = %s | shape = %s' % (t['name'], t['shape']) )

name = activation/Relu | shape = [  1 256]
name = activation_1/Softmax | shape = [ 1 10]
name = batch_normalization_v1/FusedBatchNorm | shape = [ 1 28 28  1]
name = batch_normalization_v1/FusedBatchNorm_add_param | shape = [1]
name = batch_normalization_v1/FusedBatchNorm_mul_0 | shape = [ 1 28 28  1]
name = batch_normalization_v1/FusedBatchNorm_mul_0_param | shape = [1]
name = batch_normalization_v1_1/FusedBatchNorm | shape = [ 1 14 14 64]
name = batch_normalization_v1_1/FusedBatchNorm_add_param | shape = [64]
name = batch_normalization_v1_1/FusedBatchNorm_mul_0 | shape = [ 1 14 14 64]
name = batch_normalization_v1_1/FusedBatchNorm_mul_0_param | shape = [64]
name = batch_normalization_v1_2/FusedBatchNorm | shape = [  1   7   7 128]
name = batch_normalization_v1_2/FusedBatchNorm_add_param | shape = [128]
name = batch_normalization_v1_2/FusedBatchNorm_mul_0 | shape = [  1   7   7 128]
name = batch_normalization_v1_2/FusedBatchNorm_mul_0_param | shape = [128]
name = batch_normalization_v1

## Show some weight

In [8]:
print(interp.get_tensor(12))

[[[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  ...

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 

***

## Inference

In [9]:
# Get MNIST test data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_test = x_test.astype(np.float32)
y_test = y_test.astype(np.float32)

In [10]:
interp.allocate_tensors()
input_ = interp.tensor(interp.get_input_details()[0]["index"])
output = interp.tensor(interp.get_output_details()[0]["index"])

In [11]:
result = list()

t_begin = datetime.datetime.now()
for x in x_test:
    x = np.expand_dims(np.expand_dims(x, axis=0), axis=3)
    interp.set_tensor(interp.get_input_details()[0]["index"], x)
    interp.invoke()
    
    result.append(np.argmax(output()))
    
t_end = datetime.datetime.now()

In [12]:
corr = np.sum(np.array(result) == y_test)
print('Accuracy: %d / %d' % (corr, len(x_test)))

Accuracy: 9936 / 10000


In [13]:
t_ = (t_end - t_begin)
print('Took %.3f [sec]' % (t_.seconds + t_.microseconds/1000000) )

Took 16.916 [sec]
